In [1]:
import tensorflow as tf
import numpy as np
import math, array

In [2]:
from ROOT import TVirtualFitter

Welcome to JupyROOT 6.11/01


In [3]:
fptype = tf.float64

# Fitting with TensorFlow + Minuit
First we define a few helpers to handle the estimators parameters etc. This is usually integral in programs like TensorFlowAnalysis or RooFit. Later we define a model using these tools. Finally we run the graph and minimise using Minuit


## Estimators and MC tools

### Estimators

Estimate the maximum of density function defined as a graph:
- **density** : density function
- **phsp** : phase space object (should have UniformSample method implemented)
- **size** : size of the random sample for maximum estimation
- **sess** : TF session
- **pdf**  : density graph
- **x**    : phase space placeholder used for the definition of the density function                     
- **size** : size of the random sample for maximum estimation                                            

Returns the estimated maximum of the density                                                         

In [4]:
def MaximumEstimator(density, phsp, size) :
    sample = phsp.UniformSample(size)
    return tf.reduce_max(density(sample))
def EstimateMaximum(sess, pdf, x, norm_sample) :
    pdf_data = sess.run(pdf, { x : norm_sample } )
    return np.nanmax( pdf_data )

In [5]:
class FitParameter(tf.Variable) :
    def __init__(self, name, init_value, lower_limit = 0., upper_limit = 0., step_size = 1e-6) :
        tf.Variable.__init__(self, init_value, dtype = fptype)
        self.init_value = init_value
        self.par_name = name
        self.step_size = step_size
        self.lower_limit = lower_limit
        self.upper_limit = upper_limit
        self.placeholder = tf.placeholder(self.dtype, shape=self.get_shape())
        self.update_op = self.assign(self.placeholder)
        self.prev_value = None
        self.error = 0.
        self.fitted_value = 0.
    def update(self, session, value) :
        if value != self.prev_value :
            session.run( self.update_op, { self.placeholder : value } )
            self.prev_value = value

    def floating(self) :
        return self.step_size > 0
    def randomise(self, session, minval, maxval, seed = None) :
        if seed : np.random.seed(seed)
        val = np.random.uniform(maxval, minval)
        self.init_value = val
        self.update(session, val)

### Sample Creation Tools

Create toy MC sample. To save memory, the sample is generated in "chunks" of a fixed size inside TF session, which are then concatenated.
- **sess** : TF session                                                                                  
- **pdf** : PDF graph                                                                                    
- **x** : phase space placeholder used for PDF definition                                              
- **phsp** : phase space                                                                                 
- **size** : size of the target data sample (if >0) or number of chunks (if <0)                          
- **majorant** : maximum PDF value for accept-reject method                                              
- **chunk** : chunk size                                                                                 
- **switches** : optional list of switches for component weights 
                                                                                                    
Create toy MC sample using accept-reject method for a density defined as a graph                     
- **density** : density graph
- **x** : phase space placeholder used for density graph definition
- **sample** : input uniformly distributed sample                                                       

Returns numpy array of generated points                                                              

In [6]:
def RunToyMC(sess, pdf, x, phsp, size, majorant, chunk = 200000, switches = None, seed = None) :
    first = True
    length = 0
    nchunk = 0
    phsp_sample = phsp.Filter(x)

    if seed : np.random.seed(seed)
    while length < size or nchunk < -size :
        initsample = phsp.UnfilteredSample(chunk, majorant)
        d1 = sess.run ( phsp_sample, feed_dict = { x : initsample } )
        d = CreateAcceptRejectSample(sess, pdf, x, d1)
        if switches :
            weights = []
            v = sess.run(pdf, feed_dict = { x : d } )
            for i in range(len(switches)) :
                fdict = {}
                for j in range(len(switches)) : fdict[switches[j]] = 0.
                fdict[switches[i]] = 1.
                fdict[x] = d
                v1 = sess.run( pdf, feed_dict = fdict )
                weights += [ v1/v ]
            d = np.append(d, np.transpose(np.array(weights, dtype = np.dtype('f'))), axis = 1)
        if first : data = d
        else : data = np.append(data, d, axis = 0)
        first = False
        length += len(d)
        nchunk += 1
        print "  Chunk %d, size=%d, total length=%d" % (nchunk, len(d), length)
    if size > 0 :
        return data[:size]
    else :
        return data
    
def CreateAcceptRejectSample(sess, density, x, sample) :
    p = np.transpose(np.transpose(sample)[:-1])
    r = np.transpose(sample)[-1]
    pdf_data = sess.run(density, feed_dict = {x : p})
    return p[ pdf_data > r ]

### Misc TensorFlow interfaces

In [7]:
def Sqrt(x) : return tf.sqrt(x)
def Cos(x) : return tf.cos(x)
def Integral(pdf) : return tf.reduce_mean(pdf)
def UnbinnedNLL(pdf, integral) : return -tf.reduce_sum(tf.log(pdf/integral ))

## MINUIT implementation

Perform MINUIT minimisation of the negative likelihood. 

- **sess** : TF session
- **nll** : graph for negitive likelihood to be minimised
- **feed_dict** :
- **call_limit** : call limit for MINUIT
- **gradient** : external gradient graph. If None and useGradient is not False, will be calculated internally         - **useGradient** : flag to control the use of analytic gradient while fitting:                          
    - None or False   : gradient is not used
    - True or "CHECK" : analytic gradient will be checked with finite elements, and will be used is they match
    - "FORCE"         : analytic gradient will be used regardless. 

In [8]:
cacheable_tensors = []
def RunMinuit(sess, nll, feed_dict, call_limit = 50000, useGradient = True, gradient = None, printout = 50, tmpFile = "tmp_result.txt" ) :
    global cacheable_tensors
    tfpars = tf.trainable_variables()                      # Create TF variables                           
    float_tfpars = [ p for p in tfpars if p.floating() ]   # List of floating parameters                   
    if useGradient and gradient is None :
        gradient = tf.gradients(nll, float_tfpars)            # Get analytic gradient                        

    cached_data = {}

    fetch_list = []
    for i in cacheable_tensors :
        if i not in cached_data : fetch_list += [ i ]
    feeds = dict(feed_dict)
    for i in cacheable_tensors :
        if i in cached_data : feeds[i] = cached_data[i]

    fetch_data = sess.run(fetch_list, feed_dict = feeds ) # Calculate log likelihood                       

    for i,d in zip(fetch_list, fetch_data) :
        cached_data[i] = d

    feeds = dict(feed_dict)
    for i in cacheable_tensors :
        if i in cached_data : feeds[i] = cached_data[i]                                                                                         

    def fcn(npar, gin, f, par, istatus) :                  # MINUIT fit function                           
        for i in range(len(float_tfpars)) : float_tfpars[i].update(sess, par[i])
        f[0] = sess.run(nll, feed_dict = feeds ) # Calculate log likelihood                                  
        if istatus == 2 :            # If gradient calculation is needed                                     
            dnll = sess.run(gradient, feed_dict = feeds )  # Calculate analytic gradient                       
            for i in range(len(float_tfpars)) : gin[i] = dnll[i] # Pass gradient to MINUIT                     
        fcn.n += 1
        if fcn.n % printout == 0 :
            print "  Iteration ", fcn.n, ", Flag=", istatus, " NLL=", f[0], ", pars=", sess.run(float_tfpars)

    fcn.n = 0
    minuit = TVirtualFitter.Fitter(0, len(tfpars))        # Create MINUIT instance                         
    minuit.Clear()
    minuit.SetFCN(fcn)
    arglist = array.array('d', 10*[0])    # Auxiliary array for MINUIT parameters                          

    for n,p in enumerate(float_tfpars) :  # Declare fit parameters in MINUIT                               
        step_size = p.step_size
        lower_limit = p.lower_limit
        upper_limit = p.upper_limit
        if not step_size : step_size = 1e-6
        if not lower_limit : lower_limit = 0.
        if not upper_limit : upper_limit = 0.
        minuit.SetParameter(n, p.par_name, p.init_value, step_size, lower_limit, upper_limit)

    if useGradient == True or useGradient == "CHECK" :
        minuit.ExecuteCommand("SET GRA", arglist, 0)  # Ask analytic gradient                                
    elif useGradient == "FORCE" :
        arglist[0] = 1
        minuit.ExecuteCommand("SET GRA", arglist, 1)  # Ask analytic gradient                                
    arglist[0] = call_limit                       # Set call limit                                         
    minuit.ExecuteCommand("MIGRAD", arglist, 1)   # Perform minimisation                                   

    results = {}                                  # Get fit results and update parameters                  
    for n,p in enumerate(float_tfpars) :
        p.update(sess, minuit.GetParameter(n) )
        p.fitted_value = minuit.GetParameter(n)
        p.error = minuit.GetParError(n)
        results[p.par_name] = ( p.fitted_value, p.error )

    # Get status of minimisation and NLL at the minimum                                                    
    maxlh = array.array("d", [0.])
    edm = array.array("d", [0.])
    errdef = array.array("d", [0.])
    nvpar = array.array("i", [0])
    nparx = array.array("i", [0])
    fitstatus = minuit.GetStats(maxlh, edm, errdef, nvpar, nparx)

    # return fit results                                                                                   
    results["loglh"] = maxlh[0]
    results["status"] = fitstatus
    return results

## Define FourBodyAngularPhaseSpace

Generate uniform sample makes a number of points within some phase space.
- **size** : number of _initial_ points to generate. Not all of them will fall into phase space, so the number of points in the output will be <size.
- **majorant** : if majorant>0, add 3rd dimension to the generated tensor which is uniform number from 0 to majorant. Useful for accept-reject toy MC.

Also defined is a condition and the mechanics to filter on this condition.

**Note** *it does not actually generate the sample, but returns the data flow graph for generation, which has to be run within TF session.*

In [9]:
class FourBodyAngularPhaseSpace :
    def __init__(self) :
        self.data_placeholder = self.Placeholder("data")
        self.norm_placeholder = self.Placeholder("data")

    def Inside(self, x) :
        cos1 = self.CosTheta1(x)
        cos2 = self.CosTheta2(x)
        phi  = self.Phi(x)

        inside = tf.logical_and(tf.logical_and(tf.greater(cos1, -1.), tf.less(cos1, 1.)), 
                            tf.logical_and(tf.greater(cos2, -1.), tf.less(cos2, 1.)))
        inside = tf.logical_and(inside, 
                            tf.logical_and(tf.greater(phi, 0.), tf.less(phi, 2.*math.pi ))
                           )
        return inside

    def Filter(self, x) :
        return tf.boolean_mask(x, self.Inside(x) )
        
    def UnfilteredSample(self, size, majorant = -1) :
        v = [
              np.random.uniform(-1., 1., size ).astype('d'),
              np.random.uniform(-1., 1., size ).astype('d'),
              np.random.uniform(0., 2.*math.pi, size ).astype('d')
                ]
        if majorant>0 : v += [ np.random.uniform( 0., majorant, size).astype('d') ]
        return np.transpose(np.array(v))

        
    def UniformSample(self, size, majorant = -1) :
        return self.Filter( self.UnfilteredSample(size, majorant) )

    def CosTheta1(self, sample) :
        return tf.transpose(sample)[0]

    def CosTheta2(self, sample) :
        return tf.transpose(sample)[1]

    def Phi(self, sample) :
        return tf.transpose(sample)[2]

    def Placeholder(self, name = None) :
        return tf.placeholder(fptype, shape = (None, None), name = name )

In [10]:
phsp = FourBodyAngularPhaseSpace()

FL  = FitParameter("FL" ,  0.770,  0.000, 1.000, 0.01)
AT2 = FitParameter("AT2",  0.200, -1.000, 1.000, 0.01)
S5  = FitParameter("S5" , -0.100, -1.000, 1.000, 0.01)

## Define Model

In [11]:
def model(x) :
    cosThetaK = phsp.CosTheta1(x)
    cosThetaL = phsp.CosTheta2(x)
    phi = phsp.Phi(x)

    sinThetaK = Sqrt( 1.0 - cosThetaK * cosThetaK )
    sinThetaL = Sqrt( 1.0 - cosThetaL * cosThetaL )

    sinTheta2K =  (1.0 - cosThetaK * cosThetaK)
    sinTheta2L =  (1.0 - cosThetaL * cosThetaL)

    sin2ThetaK = (2.0 * sinThetaK * cosThetaK)
    cos2ThetaL = (2.0 * cosThetaL * cosThetaL - 1.0)

    pdf  = (3.0/4.0) * (1.0 - FL ) * sinTheta2K
    pdf +=  FL * cosThetaK * cosThetaK
    pdf +=  (1.0/4.0) * (1.0 - FL) * sin2ThetaK *  cos2ThetaL
    pdf +=  (-1.0) * FL * cosThetaK * cosThetaK *  cos2ThetaL
    pdf +=  (1.0/2.0) * (1.0 - FL) * AT2 * sinTheta2K * sinTheta2L * Cos(2.0 * phi )
    pdf +=  S5 * sin2ThetaK * sinThetaL * Cos( phi )

    return pdf

## Define Graph!

### Data and shape from phasespace

In [12]:
data_ph = phsp.data_placeholder
norm_ph = phsp.norm_placeholder

### Initialise tensor flow variables

In [13]:
init = tf.global_variables_initializer()
sess = tf.Session()  
sess.run(init)

### Create sample

In [14]:
norm_sample = sess.run( phsp.UniformSample(1000000) )
majorant = EstimateMaximum(sess, model(data_ph), data_ph, norm_sample )*1.1
print "Maximum = ", majorant

Maximum =  1.71372847246


In [15]:
data_sample = RunToyMC( sess, model(data_ph), data_ph, phsp, 10000, majorant, chunk = 1000000)
norm = Integral( model(norm_ph) )
nll = UnbinnedNLL( model(data_ph), norm )

  Chunk 1, size=266793, total length=266793


### Minimize!

In [16]:
result = RunMinuit(sess, nll, { data_ph : data_sample, norm_ph : norm_sample } )
print result

{'status': 3, 'AT2': (0.04613023996759291, 0.11273485910265801), 'FL': (0.7717309583475085, 0.00857908507854227), 'S5': (-0.11507803557934504, 0.015583285157298121), 'loglh': -2333.305033661937}
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 FL           7.70000e-01  1.00000e-02    0.00000e+00  1.00000e+00
     2 AT2          2.00000e-01  1.00000e-02   -1.00000e+00  1.00000e+00
     3 S5          -1.00000e-01  1.00000e-02   -1.00000e+00  1.00000e+00
 **********
 **    1 **SET GRA
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 CHECK OF GRADIENT CALCULATION IN FCN
            PARAMETER      G(IN FCN)   G(MINUIT)  DG(MINUIT)   AGREEMENT
           1          FL -2.6863e+01 -2.6863e+01  2.1460e-05    GOOD
           2         AT2  2.4899e+01  2.4899e+01  2.8624e-07    GOOD
           3          S5  1.3282e+02  1.3282e+02  1.6291e-05    GOOD
 **********
 **    2 **MIGRAD       5e+04
 **********
 START MIGRAD MINIMIZATION. 